# Extras: stretching and uniformization

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr
import pandas as pd

import synthia as syn

import pyvinecopulib as pv

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import psychrolib as psy

In [ ]:
psy.SetUnitSystem(psy.SI)

# Generate the data
n_points = 50
TDryBulb = np.linspace(-20, 50, num=n_points)
RelHum = np.linspace(0.1, 1, num=n_points)
Pressure =  101325
grid = np.meshgrid(TDryBulb, RelHum, Pressure)

# Shape
df = pd.DataFrame([grid[0].flatten(), grid[1].flatten(), grid[2].flatten()]).T
df.columns = ['Temperature', 'RH', 'Pressure']

# Create outputs
v_GetHumRatioFromRelHum = np.vectorize(psy.GetHumRatioFromRelHum)
HumRatio = v_GetHumRatioFromRelHum(df.Temperature, df.RH, df.Pressure)

df = pd.concat([df, pd.Series(HumRatio, name='HumRatio')], axis=1)
df = df.drop(['RH'], axis=1)

h = sns.jointplot(df.Temperature, df.HumRatio, kind='scatter', stat_func=None, alpha=0.1)
h.set_axis_labels('Temperature', 'HumRatio', fontsize=16);

In [ ]:
# Our library requires xarray.Dataset
ds = df.to_xarray()
ds

In [ ]:
# Fit copula to data
parameterizer = syn.QuantileParameterizer(n_quantiles=100)
generator = syn.CopulaDataGenerator()
generator.fit(ds, copula=syn.GaussianCopula(), parameterize_by=parameterizer)

In [ ]:
# Generate samples
n_synth_samples = 10000 # we have three dims -- this is the same number of samples as true.
ds_synthetic = generator.generate(n_samples=n_synth_samples, uniformization_ratio=0, stretch_factor=1)

In [ ]:
h = sns.jointplot(ds_synthetic.Temperature, ds_synthetic.HumRatio, kind='scatter', stat_func=None, alpha=0.1)
h.set_axis_labels('Temperature', 'HumRatio', fontsize=16);